# Effects of isoproterenol

In [ ]:
using ModelingToolkit
using OrdinaryDiffEq, SteadyStateDiffEq, DiffEqCallbacks
using Plots
using CSV
using DataFrames
using Dates
using CaMKIIModel
using CaMKIIModel: ms, μM
Plots.default(lw=1.5)

## Setup model

In [ ]:
@mtkcompile sys = build_neonatal_ecc_sys()
tend = 205* 1000ms
prob = ODEProblem(sys, [], tend)
stimstart = 30* 1000ms
stimend = 120* 1000ms
alg = KenCarp47()

## Without isoproterenol

In [ ]:
@unpack Istim = sys
callback = build_stim_callbacks(Istim, stimend; period=1* 1000ms, starttime=stimstart)
@time sol = solve(prob, alg; callback)

In [ ]:
i = (sys.t / 1000, sys.vm)
plot(sol, idxs=i, tspan=(100* 1000ms, 101* 1000ms), title="Action potential", xlabel="Time (s)")

In [ ]:
plot(sol, idxs=(sys.t / 1000, [sys.Cai_sub_SR, sys.Cai_sub_SL, sys.Cai_mean]), tspan=(100* 1000ms, 101* 1000ms), title="Calcium transient", xlabel="Time (s)", ylabel="Conc. (μM)", label=["Ca (SR)" "Ca (SL)" "Ca (avg)"])

In [ ]:
plot(sol, idxs=(sys.t / 1000, sys.CaMKAct * 100), title="Active CaMKII", label=false, ylabel="Active fraction (%)", xlabel="Time (s)")

## 0.1uM isoproterenol

In [ ]:
prob2 = remake(prob, p=[sys.ISO => 0.1μM])
sol2 = solve(prob2, alg; callback)

In [ ]:
plot(sol2, idxs=(sys.t / 1000, sys.vm), tspan=(100* 1000ms, 101* 1000ms), title="Action potential", xlabel="Time (s)")

In [ ]:
plot(sol2, idxs=(sys.t / 1000, [sys.Cai_sub_SR, sys.Cai_sub_SL, sys.Cai_mean]), tspan=(100* 1000ms, 101* 1000ms), title="Calcium transcient", xlabel="Time (s)", ylabel="Conc. (μM)", label=["Ca (SR)" "Ca (SL)" "Ca (avg)"])

In [ ]:
plot(sol2, idxs=(sys.t / 1000, sys.CaMKAct * 100), title="Active CaMKII", label=false, ylabel="Active fraction (%)", xlabel="Time (s)")

## Comparison

In [ ]:
i = (sys.t / 1000, sys.Cai_mean)
tspan = (100* 1000ms, 101* 1000ms)
plot(sol, idxs=i, title="Calcium transcient", lab="ISO (-)"; tspan)
plot!(sol2, idxs=i, lab="ISO (0.1uM)", xlabel="Time (s)", ylabel="Concentration (μM)"; tspan)

In [ ]:
savefig("iso-caT.pdf")

Maximal and minimal calcium concentrations in calcium transients.

In [ ]:
ca_ctl = sol(tspan[1]:1:tspan[2], idxs=sys.Cai_mean)
println(extrema(ca_ctl))

In [ ]:
ca_iso = sol2(tspan[1]:1:tspan[2], idxs=sys.Cai_mean)
println(extrema(ca_iso))

In [ ]:
i = (sys.t / 1000, sys.CaMKAct * 100)
plot(sol, idxs=i, title="Active CaMKII", lab="ISO (-)")
plot!(sol2, idxs=i, lab="ISO (0.1uM)", ylabel="Active fraction (%)", xlabel="Time (s)")

In [ ]:
savefig("iso-camkact.pdf")

In [ ]:
i = (sys.t / 1000, sys.vm)
tspan = (100* 1000ms, 101* 1000ms)
plot(sol, idxs=i, title="Action potential", lab="ISO (-)"; tspan)
plot!(sol2, idxs=i, lab="ISO (0.1uM)", xlabel="Time (ms)", ylabel="Voltage (mV)"; tspan)

## Experimental data

In [ ]:
chemicaldf = CSV.read(joinpath(@__DIR__, "data/CaMKAR-chemical.csv"), DataFrame)
ts = Dates.value.(chemicaldf[!, "Time"]) ./ 10^9
ctl = chemicaldf[!, "Ctrl Mean"]
ctl_error = chemicaldf[!, "Ctrl SD"] ./ sqrt.(chemicaldf[!, "Ctrl N"])

In [ ]:
iso = chemicaldf[!, "isoproterenol 100nM Mean"]
iso_error = chemicaldf[!, "isoproterenol 100nM SD"] ./ sqrt.(chemicaldf[!, "isoproterenol 100nM N"])

In [ ]:
plot(ts, ctl, yerr=ctl_error, lab="Control", color=:blue, markerstrokecolor=:blue)
plot!(ts, iso, yerr=iso_error, lab="ISO 100nM", color=:red, markerstrokecolor=:red)
plot!(xlabel="Time (sec.)", ylabel="CaMKII activity (A.U.)")

In [ ]:
savefig("iso-exp.pdf")

## Runtime information

In [ ]:
using InteractiveUtils
InteractiveUtils.versioninfo()

In [ ]:
using Pkg
Pkg.status()